In [1]:
import gym
from tqdm.notebook import tqdm
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
import agents as agents
from importlib import reload

agent, ReplayBuffer, ActorCritic = agents.agent, agents.ReplayBuffer, agents.ActorCritic

def plot(frame_idx, rewards):
    
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()

In [2]:
reload(agents)

<module 'agents' from 'C:\\Users\\theod\\PycharmProjects\\Machine_Learning\\Project\\classes\\Learning\\agents.py'>

In [3]:
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
print(action_dim, state_dim)

2 4


In [4]:
agent = ActorCritic(state_dim, action_dim, alpha=0.01)
#agent = agent(state_dim, action_dim)
memory = ReplayBuffer(1000000)

BATCH_SIZE = 128
MAX_EPISODES = 5000
TAU = 0.001

In [5]:
rewards = []
mean_rewards = []
frame_idx = 0
ep_len = []
actions = np.zeros(action_dim)
np.random.seed(1)
for episodes in tqdm(range(MAX_EPISODES)):
    state = env.reset()
    episode_reward = 0
    trajectory = []
    done = False
    while not done:
        if episodes>100:
            action = agent.get_action(state)
        else:
            action = np.random.choice(action_dim)
            
        next_state, reward, done, _ = env.step(action)
        
        episode_reward += reward
        memory.push(state, action, reward, next_state, done)
        
        state = next_state
        
        frame_idx += 1 
        actions[action] += 1
        trajectory.append(int(action))
        
        if memory.__len__() > BATCH_SIZE:
            sample = memory.sample(BATCH_SIZE)
            agent.update(sample)
        if done:
            break
  
    rewards.append(episode_reward)
    mean_rewards.append(np.mean(rewards[-50:]))
        
    if frame_idx % 100:
        plot(frame_idx, mean_rewards)
        
#         if frame_idx%1000:
#             #print('\n ep_length: ',np.mean(ep_len[-50:]))
#             print('\n actions: ', trajectory)

KeyboardInterrupt: 